# Telecom Churn Case Study

###### Problem Statement

*In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of <b>15-25% annual churn rate</b>. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.*

###### Task to Perform
To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

###### Understandings

There are two main models of payment in the telecom industry 
- postpaid (customers pay a monthly/annual bill after using the services)
- prepaid (customers pay/recharge with a certain amount in advance and then use the services)

***Postpaid***: when customers want to switch to another operator, they usually inform the existing operator to terminate the services, and you directly know that this is an instance of churn.

***Prepaid***: customers who want to switch to another network can simply stop using the services without any notice, and it is hard to know whether someone has actually churned or is simply not using the services temporarily.

#### Step1: Data Exploration and Preparation

In [33]:
# Importing the required library
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning) 

***Loading Data using pandas***

In [34]:
# Reading the Telecom dataset
telecom = pd.read_csv('./telecom_churn_data.csv')

# Displaying the top few reords
telecom.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


###### Basic Information

- *Analysing the Dataset about the datatype, columns etc*

In [35]:
telecom.shape

(99999, 226)

In [36]:
telecom.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


***Findings:***

- The dataset contains 99,999 rows and 226 columns
- Out of 226 columns 179 float, 35 int and 12 object datatype

### Exploratory Data Analysis

#### Step1: Data Cleaning

- First we will drop the columns that are not useful for our analysis like the columns that contains only one value

In [37]:
# Fetching the columns that contains same date in entire column
cols_no_unique = []

for col in telecom.columns:
    if telecom[col].nunique() == 1:
        cols_no_unique.append(col)

cols_no_unique

['circle_id',
 'loc_og_t2o_mou',
 'std_og_t2o_mou',
 'loc_ic_t2o_mou',
 'last_date_of_month_6',
 'last_date_of_month_7',
 'last_date_of_month_8',
 'last_date_of_month_9',
 'std_og_t2c_mou_6',
 'std_og_t2c_mou_7',
 'std_og_t2c_mou_8',
 'std_og_t2c_mou_9',
 'std_ic_t2o_mou_6',
 'std_ic_t2o_mou_7',
 'std_ic_t2o_mou_8',
 'std_ic_t2o_mou_9']

- Dropping the above columns as it has only single value that may not be useful for our analysis

In [38]:
# Dropping the columns
telecom = telecom.drop(cols_no_unique, axis=1)
telecom.shape

(99999, 210)

### Treating Null Values

We will Start by looking at the percentage of null records in each column

In [39]:
# Percentage of null value analysis more than 0
(telecom.isna().mean() * 100)[(telecom.isna().mean() * 100) != 0]

onnet_mou_6          3.937039
onnet_mou_7          3.859039
onnet_mou_8          5.378054
onnet_mou_9          7.745077
offnet_mou_6         3.937039
                      ...    
night_pck_user_9    74.077741
fb_user_6           74.846748
fb_user_7           74.428744
fb_user_8           73.660737
fb_user_9           74.077741
Length: 152, dtype: float64

In [40]:
categorical_data = telecom.select_dtypes(include=['object'])
categorical_data.head()

,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9
0,6/21/2014,7/16/2014,8/8/2014,9/28/2014,6/21/2014,7/16/2014,8/8/2014,NaN
1,6/29/2014,7/31/2014,8/28/2014,9/30/2014,NaN,7/25/2014,8/10/2014,NaN
2,6/17/2014,7/24/2014,8/14/2014,9/29/2014,NaN,NaN,NaN,9/17/2014
3,6/28/2014,7/31/2014,8/31/2014,9/30/2014,NaN,NaN,NaN,NaN
4,6/26/2014,7/28/2014,8/9/2014,9/28/2014,6/4/2014,NaN,NaN,NaN


In [41]:
# Function to convert object to date type of the columns
def convertToDate(columns, dataset, dateFormat='%m/%d/%Y'):
    for col in columns:
        dataset[col] = pd.to_datetime(dataset[col], format=dateFormat)

In [42]:
# Convert the date type columns to datetime format
convertToDate(telecom.select_dtypes(include=['object']).columns, telecom)
telecom.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 210 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   mobile_number             int64         
 1   arpu_6                    float64       
 2   arpu_7                    float64       
 3   arpu_8                    float64       
 4   arpu_9                    float64       
 5   onnet_mou_6               float64       
 6   onnet_mou_7               float64       
 7   onnet_mou_8               float64       
 8   onnet_mou_9               float64       
 9   offnet_mou_6              float64       
 10  offnet_mou_7              float64       
 11  offnet_mou_8              float64       
 12  offnet_mou_9              float64       
 13  roam_ic_mou_6             float64       
 14  roam_ic_mou_7             float64       
 15  roam_ic_mou_8             float64       
 16  roam_ic_mou_9             float64       
 17  roam_og_mou

In [43]:
# function to split the date columns to separate date, month and years column
def splitDateColumns(columns, dataset):
    for col in columns:
        column_mnt = col + '_mnt'
        column_yr = col + '_yr'
        column_d = col + '_d'
        dataset[column_mnt] = dataset[col].dt.month
        dataset[column_yr] = dataset[col].dt.year
        dataset[column_d] = dataset[col].dt.day

In [44]:
splitDateColumns(telecom.select_dtypes(include=['datetime']).columns, telecom)
telecom.head()

,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,...,date_of_last_rech_data_6_d,date_of_last_rech_data_7_mnt,date_of_last_rech_data_7_yr,date_of_last_rech_data_7_d,date_of_last_rech_data_8_mnt,date_of_last_rech_data_8_yr,date_of_last_rech_data_8_d,date_of_last_rech_data_9_mnt,date_of_last_rech_data_9_yr,date_of_last_rech_data_9_d
0,7000842753,197.385,214.816,213.803,21.100,NaN,NaN,0.00,NaN,NaN,...,21.0,7.0,2014.0,16.0,8.0,2014.0,8.0,NaN,NaN,NaN
1,7001865778,34.047,355.074,268.321,86.285,24.11,78.68,7.68,18.34,15.74,...,NaN,7.0,2014.0,25.0,8.0,2014.0,10.0,NaN,NaN,NaN
2,7001625959,167.690,189.058,210.226,290.714,11.54,55.24,37.26,74.81,143.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2014.0,17.0
3,7001204172,221.338,251.102,508.054,389.500,99.91,54.39,310.98,241.71,123.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7000142493,261.636,309.876,238.174,163.426,50.31,149.44,83.89,58.78,76.96,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
telecom_data = telecom.drop(columns=telecom.select_dtypes(include=['datetime']).columns, axis=1)

In [46]:
telecom_data.shape

(99999, 226)

In [47]:
telecom_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to date_of_last_rech_data_9_d
dtypes: float64(192), int64(34)
memory usage: 172.4 MB
